## World Cities

source: [世界の百万都市の位置データ Location Data of Megacities - ASTI アマノ技研](https://amano-tec.com/data/megacities.html)

In [1]:
import json
import re
from collections import defaultdict

import pandas as pd
import jaconv

In [2]:
df = pd.read_csv("./asti-dats59r03wm/s59r03megacities_utf8.csv", sep="\t")
df.head().T

,0,1,2,3,4
country_code,AZ,AF,US,US,US
name_jp,アゼルバイジャン共和国,アフガニスタン・イスラム共和国,アメリカ合衆国,アメリカ合衆国,アメリカ合衆国
name_jps,アゼルバイジャン,アフガニスタン,アメリカ（米国）,アメリカ（米国）,アメリカ（米国）
name_en,Republic of Azerbaijan,Islamic Republic of Afghanistan,United States of America,United States of America,United States of America
name_ens,Azerbaijan,Afghanistan,United States,United States,United States
capital_jp,バクー,カブール,サンアントニオ,サンディエゴ,シカゴ
capital_en,Baku,Kabul,San Antonio,San Diego,Chicago
iscapital,1,1,0,0,0
lat,40.36704,34.521096,29.424587,32.721944,41.883823
lon,49.832039,69.173672,-98.495145,-117.171918,-87.632078


In [3]:
def remove_bracket(name):
        if "（" in name:
            parts = name.split("（")
            assert len(parts) == 2
            assert parts[1][-1] == "）"
            return parts[0]
        return name

In [4]:
hiragana_pattern = r'^[\u3040-\u309F]+$'
katakana_pattern = r'^[\u30A0-\u30FF]+$'

reading_dict = {
    "東京": "とうきょう",
    "大阪": "おおさか",
    "川崎": "かわさき",
    "京都": "きょうと",
    "神戸": "こうべ",
    "札幌": "さっぽろ",
    "仙台": "せんだい",
    "名古屋": "なごや",
    "広島": "ひろしま",
    "福岡": "ふくおか",
    "横浜": "よこはま"
}

def is_katakana(name):
    return re.match(katakana_pattern, name)

def name2reading(name):
    if re.match(hiragana_pattern, name):
        return name
    elif re.match(katakana_pattern, name):
        return jaconv.kata2hira(name)

    # ひらがな, カタカナ以外
    if "（" in name:
        parts = name.split("（")
        assert len(parts) == 2
        assert parts[1][-1] == "）"
        katakana = parts[1][:-1]
        return jaconv.kata2hira(katakana)

    assert name in reading_dict, name
    return reading_dict[name]

In [5]:
# 長音, 促音を考慮
def get_last_char(reading):
    if reading[-1] == "ー":
        return get_last_char(reading[:-1])
    if reading[-1] == "ぁ":
        return "あ"
    if reading[-1] == "ぃ":
        return "い"
    if reading[-1] == "ぅ":
        return "う"
    if reading[-1] == "ぇ":
        return "え"
    if reading[-1] == "ゃ":
        return "や"
    if reading[-1] =="ゅ":
        return "ゆ"
    return reading[-1]

In [6]:
cities = []

for _, row in df.iterrows():
    d = row.to_dict()
    
    # 特殊なケースをスキップ - 「ダマスカス」自体は別途含まれる
    if d["capital_jp"] == "ダマスカス郊外": continue
        
    # 🥳
    if d["capital_jp"] == "ウンジャメナ": d["capital_jp"] = "ンジャメナ"
        
    name = remove_bracket(d["capital_jp"])
    reading = name2reading(d["capital_jp"])
    
    item = {
        "name": name,
        "reading": reading,
        "country": d["name_jp"],
        "population": d["pop"],
        "coordinates": [d["lon"], d["lat"]],
        "shiritori": {
            "first": reading[0],
            "last": get_last_char(reading),
        }
    }
    
    if not is_katakana(name):
        item["notKatakana"] = True
    
    cities.append(item)

[d for d in cities if d["name"] == "ンジャメナ"]

[{'name': 'ンジャメナ',
  'reading': 'んじゃめな',
  'country': 'チャド共和国',
  'population': 1521882,
  'coordinates': [15.0448322, 12.1052915],
  'shiritori': {'first': 'ん', 'last': 'な'}}]

### deadend

In [7]:
shiritori_first_dict = defaultdict(list)
shiritori_last_dict = defaultdict(list)

for city in cities:
    shiritori_first_dict[city["shiritori"]["first"]].append(city)
    shiritori_last_dict[city["shiritori"]["last"]].append(city)

In [8]:
set(shiritori_last_dict.keys()) - set(shiritori_first_dict.keys())

{'づ'}

In [9]:
shiritori_last_dict["づ"]

[{'name': '菏沢',
  'reading': 'ほーづー',
  'country': '中華人民共和国',
  'population': 1280031,
  'coordinates': [115.4738228, 35.2348208],
  'shiritori': {'first': 'ほ', 'last': 'づ'},
  'notKatakana': True}]

In [10]:
shiritori_last_dict["ず"]

[{'name': 'アフヴァーズ',
  'reading': 'あふゔぁーず',
  'country': 'イラン・イスラム共和国',
  'population': 1184788,
  'coordinates': [48.6629412, 31.3204333],
  'shiritori': {'first': 'あ', 'last': 'ず'}},
 {'name': 'シーラーズ',
  'reading': 'しーらーず',
  'country': 'イラン・イスラム共和国',
  'population': 1565572,
  'coordinates': [52.5429486, 29.6182001],
  'shiritori': {'first': 'し', 'last': 'ず'}},
 {'name': 'タブリーズ',
  'reading': 'たぶりーず',
  'country': 'イラン・イスラム共和国',
  'population': 1558693,
  'coordinates': [46.2956786, 38.0736806],
  'shiritori': {'first': 'た', 'last': 'ず'}},
 {'name': 'フェズ',
  'reading': 'ふぇず',
  'country': 'モロッコ王国',
  'population': 1249416,
  'coordinates': [-5.016249, 34.034446],
  'shiritori': {'first': 'ふ', 'last': 'ず'}}]

In [11]:
shiritori_first_dict["ず"]

[{'name': '淄博',
  'reading': 'ずーぼー',
  'country': '中華人民共和国',
  'population': 2817479,
  'coordinates': [118.0488091, 36.813085],
  'shiritori': {'first': 'ず', 'last': 'ぼ'},
  'notKatakana': True},
 {'name': '資陽',
  'reading': 'ずーやん',
  'country': '中華人民共和国',
  'population': 1016034,
  'coordinates': [104.6251845, 30.1316754],
  'shiritori': {'first': 'ず', 'last': 'ん'},
  'notKatakana': True},
 {'name': '鄭州',
  'reading': 'ずぇんぢょう',
  'country': '中華人民共和国',
  'population': 2589387,
  'coordinates': [113.6193223, 34.7477857],
  'shiritori': {'first': 'ず', 'last': 'う'},
  'notKatakana': True},
 {'name': '鄒城',
  'reading': 'ずぉうちょん',
  'country': '中華人民共和国',
  'population': 1101003,
  'coordinates': [117.0016364, 35.4046683],
  'shiritori': {'first': 'ず', 'last': 'ん'},
  'notKatakana': True},
 {'name': '中山',
  'reading': 'ずぉんしゃん',
  'country': '中華人民共和国',
  'population': 2363322,
  'coordinates': [113.3881505, 22.5196006],
  'shiritori': {'first': 'ず', 'last': 'ん'},
  'notKatakana': True}]

In [12]:
[(i, city) for i, city in enumerate(cities) if city["shiritori"]["last"] == "づ"]

[(340,
  {'name': '菏沢',
   'reading': 'ほーづー',
   'country': '中華人民共和国',
   'population': 1280031,
   'coordinates': [115.4738228, 35.2348208],
   'shiritori': {'first': 'ほ', 'last': 'づ'},
   'notKatakana': True})]

In [13]:
cities[340]["shiritori"]["last"] = "ず"

In [14]:
cities[340]

{'name': '菏沢',
 'reading': 'ほーづー',
 'country': '中華人民共和国',
 'population': 1280031,
 'coordinates': [115.4738228, 35.2348208],
 'shiritori': {'first': 'ほ', 'last': 'ず'},
 'notKatakana': True}

### output

In [15]:
with open("../static/cities.json", "w") as fp:
    json.dump(cities, fp, ensure_ascii=False, indent=2)

In [16]:
!ls -lh ../static/cities.json

-rw-r--r--  1 sorami  staff   141K Jul 17 17:23 ../static/cities.json


In [17]:
!head -20 ../static/cities.json

[
  {
    "name": "バクー",
    "reading": "ばくー",
    "country": "アゼルバイジャン共和国",
    "population": 2285273,
    "coordinates": [
      49.8320385,
      40.3670397
    ],
    "shiritori": {
      "first": "ば",
      "last": "く"
    }
  },
  {
    "name": "カブール",
    "reading": "かぶーる",
    "country": "アフガニスタン・イスラム共和国",
    "population": 4775074,


## Stats

### long names

In [18]:
sorted([d["name"] for d in cities], reverse=True, key=lambda x: len(x))[:10]

['ナコーンシータンマラート',
 'ナコーンラーチャシーマー',
 'ピンプリ・チンチワッド',
 'ウボンラーチャターニー',
 'ヴィシャーカパトナム',
 'サントドミンゴエステ',
 'アグアスカリエンテス',
 'サンクトペテルブルク',
 'アウランガーバード',
 'ガーズィヤーバード']

### counts

In [19]:
shiritori_first_dict = defaultdict(list)
shiritori_last_dict = defaultdict(list)

for city in cities:
    shiritori_first_dict[city["shiritori"]["first"]].append(city)
    shiritori_last_dict[city["shiritori"]["last"]].append(city)

In [20]:
pd.DataFrame([(k, len(v)) for k, v in shiritori_first_dict.items()], columns=["first", "count"])\
    .sort_values("count", ascending=False)\
    .reset_index(drop=True)\
    .head(10)

,first,count
0,あ,29
1,し,28
2,ち,23
3,か,21
4,さ,20
5,は,19
6,ふ,18
7,ば,17
8,ま,16
9,な,15


In [21]:
pd.DataFrame([(k, len(v)) for k, v in shiritori_last_dict.items()], columns=["last", "count"])\
    .sort_values("count", ascending=False)\
    .reset_index(drop=True)\
    .head(10)

,last,count
0,ん,149
1,う,44
2,る,37
3,い,27
4,ど,21
5,ら,17
6,す,17
7,と,16
8,く,13
9,り,12


### no incoming cities

In [22]:
set(shiritori_first_dict.keys()) - set(shiritori_last_dict.keys())

{'が', 'け', 'せ', 'そ', 'ぢ', 'つ', 'ぴ', 'ふ', 'ぺ', 'ほ', 'ゔ'}

In [23]:
[(city["name"], city["reading"]) for city in cities if city["shiritori"]["first"] in set(shiritori_first_dict.keys()) - set(shiritori_last_dict.keys())]

[('フィラデルフィア', 'ふぃらでるふぃあ'),
 ('フェニックス', 'ふぇにっくす'),
 ('ヴァドーダラー', 'ゔぁどーだらー'),
 ('ヴィシャーカパトナム', 'ゔぃしゃーかぱとなむ'),
 ('ヴィジャヤワーダ', 'ゔぃじゃやわーだ'),
 ('ガーズィヤーバード', 'がーずぃやーばーど'),
 ('ピンプリ・チンチワッド', 'ぴんぷり・ちんちわっど'),
 ('ファリーダーバード', 'ふぁりーだーばーど'),
 ('ソウル', 'そうる'),
 ('ピョンヤン', 'ぴょんやん'),
 ('フリータウン', 'ふりーたうん'),
 ('ホムス', 'ほむす'),
 ('ピキン', 'ぴきん'),
 ('ソンクラー', 'そんくらー'),
 ('北京', 'ぺきん'),
 ('吉林', 'ぢーりん'),
 ('諸曁', 'ぢゅーじー'),
 ('鍾祥', 'ぢょんしゃん'),
 ('自貢', 'つーごん'),
 ('慈渓', 'つーしー'),
 ('成都', 'つぇんどぅー'),
 ('重慶', 'つぉんちん'),
 ('邳州', 'ぴーぢょう'),
 ('平度', 'ぴんどぅ'),
 ('淮南', 'ふぁいなん'),
 ('撫順', 'ふーしゅん'),
 ('福州', 'ふーぢょう'),
 ('撫州', 'ふーぢょう'),
 ('湖州', 'ふーぢょう'),
 ('福清', 'ふーちん'),
 ('富陽', 'ふーやん'),
 ('豊城', 'ふぉんちょん'),
 ('呼和浩特', 'ふふほと'),
 ('淮安', 'ほぁいあん'),
 ('化州', 'ほあちょう'),
 ('合川', 'ほーちゅあん'),
 ('菏沢', 'ほーづー'),
 ('香港', 'ほんこん'),
 ('ケルン', 'けるん'),
 ('ヴァン', 'ゔぁん'),
 ('ガジアンテップ', 'がじあんてっぷ'),
 ('仙台', 'せんだい'),
 ('福岡', 'ふくおか'),
 ('ファイサラバード', 'ふぁいさらばーど'),
 ('ペシャーワル', 'ぺしゃーわる'),
 ('ケソンシティ', 'けそんしてぃ'),
 ('フォルタレザ', 'ふぉるたれざ'),
 ('ソフィア', 'そふぃあ'),
 ('ホーチミン', 'ほーちみん'),
 ('ケ